In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
from statistics import mean
#import altair as alt
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

packageNames = ('ggplot2', 'tidyverse','rayshader','rgl','DiagrammeR')
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

packnames_to_install = [a for a in packageNames if not rpackages.isinstalled(a)]

if len(packnames_to_install) > 0:
    utils.install_packages(StrVector(packnames_to_install))


data = pd.read_csv("C:/Users/Nehal/JupyterNotebooks/decentraland.csv")
df = pd.DataFrame(data)

# only keep relevant columns
df = df [['x','y', 'last_sale_timestamp', 'current_rate_pricemana',
              'last_sale_transaction_timestamp', 'last_sale_transaction_created_timestamp']]

# Create date field
df['transaction_date'] = pd.to_datetime(df['last_sale_transaction_timestamp']).dt.date
df['transaction_date_created'] = pd.to_datetime(df['last_sale_transaction_created_timestamp']).dt.date

#### Calculate average price for a given area #### 
# first add columns for ranges for average
df['x_range_min'] = df["x"] - 20 
df['x_range_max'] = df["x"] + 20 
df['y_range_min'] = df["y"] - 20 
df['y_range_max'] = df["y"] + 20 


# Create function that calculates the average price
#def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
 #   current_rate_pricemana = mean([x_min,x_max,y_min,y_max])
  #  return current_rate_pricemana
print(df)

2022-02-13 14:01:47.250 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


        x    y  last_sale_timestamp  current_rate_pricemana  \
0      47   47  2021-02-15T22:31:33                  9000.0   
1     148  -33  2020-11-26T22:08:08                  8000.0   
2     148  -32  2019-01-25T16:16:24                 10685.0   
3     148  -31  2019-02-01T17:11:03                 14850.0   
4     148  -30  2019-01-11T07:57:03                 10999.0   
...   ...  ...                  ...                     ...   
6339   16   88  2020-08-08T23:00:48                    10.0   
6340   17 -103  2020-09-21T14:02:20                     3.0   
6341   17  -38  2020-07-05T21:43:35                     3.2   
6342   18 -105  2020-09-26T21:06:37                     3.0   
6343  149  -44  2020-10-24T21:13:15                     1.0   

     last_sale_transaction_timestamp last_sale_transaction_created_timestamp  \
0                2021-02-15T22:31:33              2021-02-15T22:32:02.303718   
1                2020-11-26T22:08:08              2020-11-26T22:08:36.556212   
2  

In [2]:
%load_ext rpy2.ipython

C:\Users\Nehal\anaconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [10]:
%%R

library(tidyverse)
area_avg_price_fun_eth = function(x_min,x_max,y_min,y_max){
  mean(filter(df, x >= x_min, x <= x_max, y >= y_min, y <= y_max)$current_rate_pricemana, na.rm=T)
}


#df = mutate(df, area_avg_price = pmap(list(x_range_min, x_range_max, y_range_min, y_range_max), area_avg_price_fun_eth))

In [68]:
df = df.groupby(["x_range_min","x_range_max","y_range_min","y_range_max"])["current_rate_pricemana"].mean().to_frame("area_avg_price")
print(df)

KeyError: 'Column not found: current_rate_pricemana'

In [48]:
def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
    area_price = df['current_rate_pricemana'].apply('x_min','x_max','y_min','y_max')
    area_avg_price = area_price.mean()
    return area_avg_price

#df['area_avg_price'] = df['current_rate_pricemana'].area_avg_price_fun_eth('x_range_min','x_range_max','y_range_min','y_range_max')


In [55]:
#df['area_avg_price'] = area_avg_price_fun_eth('x_range_min','x_range_max','y_range_min','y_range_max')

df['area_avg_price'] = area_avg_price_fun_eth('x','y','x_range_min','y')

#df['area_avg_price'] = df.apply(area_avg_price_fun_eth('x_range_min','x_range_max','y_range_min','y_range_max'))
#df['current_rate_pricemana'].area_avg_price_fun_eth('x_range_min','x_range_max','y_range_min','y_range_max')
#tmp = tmp.groupby(["x","y"])["total_price_usd"].mean().to_frame("avg_price_usd").reset_index()
#tmp

TypeError: apply() takes from 2 to 4 positional arguments but 5 were given

In [13]:
def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
    area_avg_price = df.apply(lambda row : df['current_rate_pricemana'](row['x_min'],row['x_max'], row['y_min'],row['y_max']).mean(), axis=1)
    return area_avg_price

df['area_avg_price'] = df['current_rate_pricemana'].apply(area_avg_price_fun_eth('x_range_min','x_range_max','y_range_min','y_range_max'))


KeyError: 'x_min'

In [85]:
#df['area_avg_price'] = df['current_rate_pricemana'].apply(lambda row: df['current_rate_pricemana'](row['x_range_min'],row['x_range_max'], row['y_range_min'],row['y_range_max']).mean())
#(["x_range_min","x_range_max","y_range_min","y_range_max"])

#area_avg_price = map(list(area_avg_price_fun_eth('x_range_min', 'x_range_max', 'y_range_min', 'y_range_max')))


#df['area_avg_price']= (df['current_rate_pricemana'].apply('x_range_min', 'x_range_max', 'y_range_min', 'y_range_max')).mean()


In [33]:

def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
    area_avg_price = df.apply(lambda row : df['current_rate_pricemana'](row['x_min'],row['x_max'], row['y_min'],row['y_max']).mean(), axis=1)
    return area_avg_price




In [86]:
#df['area_avg_price'] = df['current_rate_pricemana'].apply(lambda row: area_avg_price_fun_eth(row['x_range_min'],row['x_range_max'], row['y_range_min'],row['y_range_max']))
#df['area_avg_price'] = df['current_rate_pricemana'].apply(lambda row: area_avg_price_fun_eth(), df['area_avg_price'])

#df['area_avg_price'] = df['current_rate_pricemana'].apply(lambda row: df['current_rate_pricemana'](row['x_min'],row['x_max'], row['y_min'],row['y_max']).mean())


In [25]:
df['area_avg_price'] = df.map(["x_range_min","x_range_max","y_range_min","y_range_max"])["current_rate_pricemana"].mean()
def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
    area_avg_price = df.apply(lambda row : df['current_rate_pricemana'](row['x_min'],row['x_max'], row['y_min'],row['y_max']).mean(), axis=1)
    return area_avg_price



#df['income']=list(map(lambda x: int(x+x*0.2),df['income']))

#df['area_avg_price'] = list(map(lambda row: area_avg_price_fun_eth(), df['area_avg_price']))

#df['area_avg_price'] = df.apply(lambda row : area_avg_price_fun_eth(row['x_range_min'],row['x_range_max'], row['y_range_min'],row['y_range_max']), axis=1)
#range_max_min = ['x_range_min','x_range_max','y_range_min','y_range_max']
#area_avg_price = map(area_avg_price_fun_eth,range_max_min)
#df['area_avg_price'] = df.apply(area_avg_price_fun_eth, axis=1, result_type='expand')
#df['area_avg_price'] = df['current_rate_pricemana'].apply(lambda row : area_avg_price_fun_eth(row['x_range_min'],row['x_range_max'], row['y_range_min'],row['y_range_max']))


df = df.drop_duplicates()

KeyError: 'x_min'

In [ ]:
df['area_avg_price'] = 

In [3]:
df.head()

,x,y,last_sale_timestamp,current_rate_pricemana,last_sale_transaction_timestamp,last_sale_transaction_created_timestamp,transaction_date,transaction_date_created,x_range_min,x_range_max,y_range_min,y_range_max
0,47,47,2021-02-15T22:31:33,9000.0,2021-02-15T22:31:33,2021-02-15T22:32:02.303718,2021-02-15,2021-02-15,27,67,27,67
1,148,-33,2020-11-26T22:08:08,8000.0,2020-11-26T22:08:08,2020-11-26T22:08:36.556212,2020-11-26,2020-11-26,128,168,-53,-13
2,148,-32,2019-01-25T16:16:24,10685.0,2019-01-25T16:16:24,2019-01-25T16:18:10.536554,2019-01-25,2019-01-25,128,168,-52,-12
3,148,-31,2019-02-01T17:11:03,14850.0,2019-02-01T17:11:03,2019-02-01T17:13:29.057672,2019-02-01,2019-02-01,128,168,-51,-11
4,148,-30,2019-01-11T07:57:03,10999.0,2019-01-11T07:57:03,2019-01-11T07:58:58.549844,2019-01-11,2019-01-11,128,168,-50,-10


In [6]:
#Min and max values for Transaction Date Range
oldest = df['transaction_date'].min() # Earliest date
latest = df['transaction_date'].max() # Latest date

In [7]:
print(oldest)
print(latest)

2018-03-19
2021-02-16


In [78]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
from statistics import mean
#import altair as alt
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

packageNames = ('ggplot2', 'tidyverse','rayshader','rgl','DiagrammeR')
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

packnames_to_install = [a for a in packageNames if not rpackages.isinstalled(a)]

if len(packnames_to_install) > 0:
    utils.install_packages(StrVector(packnames_to_install))


data = pd.read_csv("C:/Users/Nehal/JupyterNotebooks/decentraland.csv")
df = pd.DataFrame(data)

# only keep relevant columns
df = df [['x','y', 'last_sale_timestamp', 'current_rate_pricemana',
              'last_sale_transaction_timestamp', 'last_sale_transaction_created_timestamp']]

# Create date field
df['transaction_date'] = pd.to_datetime(df['last_sale_transaction_timestamp']).dt.date
df['transaction_date_created'] = pd.to_datetime(df['last_sale_transaction_created_timestamp']).dt.date

#### Calculate average price for a given area #### 
# first add columns for ranges for average
df['x_range_min'] = df["x"] - 20 
df['x_range_max'] = df["x"] + 20 
df['y_range_min'] = df["y"] - 20 
df['y_range_max'] = df["y"] + 20 

# Create function that calculates the average price
def area_avg_price_fun_eth(x_min, x_max, y_min, y_max):
    area_avg_price = 'current_rate_pricemana'.mean([x_min,x_max,y_min,y_max])
    return area_avg_price

#df['area_avg_price'] = df.apply(lambda row : area_avg_price_fun_eth(row['x_range_min'],row['x_range_max'], row['y_range_min'],row['y_range_max']), axis=1)
range_max_min = ['x_range_min','x_range_max','y_range_min','y_range_max']
area_avg_price = map(area_avg_price_fun_eth,range_max_min)

df = df.drop_duplicates()

print(df)

2022-02-13 15:27:40.578 WARNING rpy2.rinterface_lib.callbacks: R[write to console]: Warning:
2022-02-13 15:27:40.580 WARNING rpy2.rinterface_lib.callbacks: R[write to console]:  failed to download mirrors file (cannot open destfile 'C:\Users\Nehal\AppData\Local\Temp\RtmpqSPUUc\file5c4c78d215ec', reason 'No such file or directory'); using local file 'C:/Program Files/R/R-4.1.2/doc/CRAN_mirrors.csv'



        x    y  last_sale_timestamp  current_rate_pricemana  \
0      47   47  2021-02-15T22:31:33                  9000.0   
1     148  -33  2020-11-26T22:08:08                  8000.0   
2     148  -32  2019-01-25T16:16:24                 10685.0   
3     148  -31  2019-02-01T17:11:03                 14850.0   
4     148  -30  2019-01-11T07:57:03                 10999.0   
...   ...  ...                  ...                     ...   
3732  148 -110  2020-12-06T02:05:16                  6999.0   
3733  148  -55  2019-02-17T01:14:38                 13190.0   
3734  148  -54  2019-02-10T10:26:36                 11500.0   
3735  148  -36  2020-12-09T16:26:02                  8000.0   
3736  148  -35  2020-12-09T16:42:05                  8000.0   

     last_sale_transaction_timestamp last_sale_transaction_created_timestamp  \
0                2021-02-15T22:31:33              2021-02-15T22:32:02.303718   
1                2020-11-26T22:08:08              2020-11-26T22:08:36.556212   
2  

In [83]:
df['area_avg_price'] = df['current_rate_pricemana'] * 2.96

In [84]:
print(df)

        x    y  last_sale_timestamp  current_rate_pricemana  \
0      47   47  2021-02-15T22:31:33                  9000.0   
1     148  -33  2020-11-26T22:08:08                  8000.0   
2     148  -32  2019-01-25T16:16:24                 10685.0   
3     148  -31  2019-02-01T17:11:03                 14850.0   
4     148  -30  2019-01-11T07:57:03                 10999.0   
...   ...  ...                  ...                     ...   
3732  148 -110  2020-12-06T02:05:16                  6999.0   
3733  148  -55  2019-02-17T01:14:38                 13190.0   
3734  148  -54  2019-02-10T10:26:36                 11500.0   
3735  148  -36  2020-12-09T16:26:02                  8000.0   
3736  148  -35  2020-12-09T16:42:05                  8000.0   

     last_sale_transaction_timestamp last_sale_transaction_created_timestamp  \
0                2021-02-15T22:31:33              2021-02-15T22:32:02.303718   
1                2020-11-26T22:08:08              2020-11-26T22:08:36.556212   
2  